In [4]:
import pandas as pd
from datetime import datetime
import numpy as np
import scipy as sp
import sklearn
import sys

from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from scipy import linalg

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import pickle
import string
import matplotlib.pyplot as plt

In [27]:
import pandas as pd
import numpy as np

# 크롤링한 데이터 불러오기

data = pd.read_csv('2019술집.csv', encoding='utf-8-sig')
data

,Unnamed: 0,time,review,comment,like,url
0,0,2019. 1. 13. 21:35,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ...",[],8,https://blog.naver.com/14thmay/221441149660
1,1,2019. 1. 5. 13:31,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...,[],15,https://blog.naver.com/nodaybuttoday/221434831902
2,2,2019. 1. 27. 22:20,"술집 창원 애월애, 인스타 감성 술집 텔스 ...",[],4,https://blog.naver.com/seyyyoung/221452442833
3,3,2019. 1. 16. 8:20,"술 집, 대구 동성로 술집 : 인스타 감성 술집 어쩌면 ...",[],17,https://blog.naver.com/chuuny_eunn/221443164007
4,4,2019. 1. 31. 14:39,tasty food [홍대 김치찌개 맛집] 맛있고 분위기...,[],10,https://blog.naver.com/blancji03/221455594209
...,...,...,...,...,...,...
21236,21236,2019. 12. 2. 18:20,맛맛 강변역 술집 낭만식객 분위기 좋은 이자카야 유...,[],12,https://blog.naver.com/dldbwk0812/221724901082
21237,21237,2019. 12. 16. 18:30,먹어 본 . 그 맛 이태원 라운지바 데이앤나잇 분위기 좋은 술...,[],5,https://blog.naver.com/jjangmi95/221733856620
21238,21238,2019. 12. 14. 15:44,맛집 대구 술집/ 대구펍]분위기 좋은 삼덕동술집 퍼스(pert...,[],6,https://blog.naver.com/gidtnr5515/221737168600
21239,21239,2019. 12. 9. 16:29,애정하는✍ [신촌] 분위기 있는 조용한 카페&술집 #에어플레인...,[],33,https://blog.naver.com/dndbajrwk1/221731786787


In [28]:
# 제거 대상 단어
Stopwords = ['제공받아','제공받았','지원받아','지원받았','해당업체','해시태그','업체로부터',
             '체험후기','무상지급받은','무상지급','카카오톡플러스친구','톡톡하세요','네이버톡톡',
             '네이버 톡톡','무상지원','카카오톡 플러스친구','스토리링크','카카오 채널',
            ]

In [29]:
# stopwords가 들어간 글은 삭제
data = data[~data['review'].str.contains('|'.join(Stopwords))]
data = data.reset_index(drop=True)
data

,Unnamed: 0,time,review,comment,like,url
0,0,2019. 1. 13. 21:35,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ...",[],8,https://blog.naver.com/14thmay/221441149660
1,1,2019. 1. 5. 13:31,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...,[],15,https://blog.naver.com/nodaybuttoday/221434831902
2,2,2019. 1. 27. 22:20,"술집 창원 애월애, 인스타 감성 술집 텔스 ...",[],4,https://blog.naver.com/seyyyoung/221452442833
3,3,2019. 1. 16. 8:20,"술 집, 대구 동성로 술집 : 인스타 감성 술집 어쩌면 ...",[],17,https://blog.naver.com/chuuny_eunn/221443164007
4,4,2019. 1. 31. 14:39,tasty food [홍대 김치찌개 맛집] 맛있고 분위기...,[],10,https://blog.naver.com/blancji03/221455594209
...,...,...,...,...,...,...
20577,21236,2019. 12. 2. 18:20,맛맛 강변역 술집 낭만식객 분위기 좋은 이자카야 유...,[],12,https://blog.naver.com/dldbwk0812/221724901082
20578,21237,2019. 12. 16. 18:30,먹어 본 . 그 맛 이태원 라운지바 데이앤나잇 분위기 좋은 술...,[],5,https://blog.naver.com/jjangmi95/221733856620
20579,21238,2019. 12. 14. 15:44,맛집 대구 술집/ 대구펍]분위기 좋은 삼덕동술집 퍼스(pert...,[],6,https://blog.naver.com/gidtnr5515/221737168600
20580,21239,2019. 12. 9. 16:29,애정하는✍ [신촌] 분위기 있는 조용한 카페&술집 #에어플레인...,[],33,https://blog.naver.com/dndbajrwk1/221731786787


In [30]:
ix = data['review'].str.len() > 3000
data = data.loc[~ix]
data = data.reset_index(drop=True)
data

,Unnamed: 0,time,review,comment,like,url
0,0,2019. 1. 13. 21:35,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ...",[],8,https://blog.naver.com/14thmay/221441149660
1,1,2019. 1. 5. 13:31,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...,[],15,https://blog.naver.com/nodaybuttoday/221434831902
2,2,2019. 1. 27. 22:20,"술집 창원 애월애, 인스타 감성 술집 텔스 ...",[],4,https://blog.naver.com/seyyyoung/221452442833
3,5,2019. 1. 21. 17:00,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 ...,[],23,https://blog.naver.com/hongki1990/221447452932
4,7,2019. 1. 5. 2:57,식당 여수 학동 술집 추천 / 인스타 핫플 파흐베륵 ...,[],7,https://blog.naver.com/soso__log/221434626247
...,...,...,...,...,...,...
13392,21230,2019. 12. 30. 23:58,냐미 동래 분위기 좋은 술집 : 주화담 강추 ! 리...,[],10,https://blog.naver.com/ksrsol/221754718804
13393,21231,2019. 12. 29. 12:37,먹기위해 산다 합정동 학쌀롱 - 분위기 좋은 합정역 술집 !!...,[],11,https://blog.naver.com/kyungbling/221753017814
13394,21232,2019. 12. 29. 10:46,in 서울 [성수 술집] 맛도 분위기도 좋은 이자카야 용싸롱 ...,[],11,https://blog.naver.com/soar2high/221752944616
13395,21237,2019. 12. 16. 18:30,먹어 본 . 그 맛 이태원 라운지바 데이앤나잇 분위기 좋은 술...,[],5,https://blog.naver.com/jjangmi95/221733856620


In [31]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus


In [32]:
data

,Unnamed: 0,time,review,comment,like,url
0,0,2019. 1. 13. 21:35,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ...",[],8,https://blog.naver.com/14thmay/221441149660
1,1,2019. 1. 5. 13:31,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...,[],15,https://blog.naver.com/nodaybuttoday/221434831902
2,2,2019. 1. 27. 22:20,"술집 창원 애월애, 인스타 감성 술집 텔스 ...",[],4,https://blog.naver.com/seyyyoung/221452442833
3,5,2019. 1. 21. 17:00,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 ...,[],23,https://blog.naver.com/hongki1990/221447452932
4,7,2019. 1. 5. 2:57,식당 여수 학동 술집 추천 / 인스타 핫플 파흐베륵 ...,[],7,https://blog.naver.com/soso__log/221434626247
...,...,...,...,...,...,...
13392,21230,2019. 12. 30. 23:58,냐미 동래 분위기 좋은 술집 : 주화담 강추 ! 리...,[],10,https://blog.naver.com/ksrsol/221754718804
13393,21231,2019. 12. 29. 12:37,먹기위해 산다 합정동 학쌀롱 - 분위기 좋은 합정역 술집 !!...,[],11,https://blog.naver.com/kyungbling/221753017814
13394,21232,2019. 12. 29. 10:46,in 서울 [성수 술집] 맛도 분위기도 좋은 이자카야 용싸롱 ...,[],11,https://blog.naver.com/soar2high/221752944616
13395,21237,2019. 12. 16. 18:30,먹어 본 . 그 맛 이태원 라운지바 데이앤나잇 분위기 좋은 술...,[],5,https://blog.naver.com/jjangmi95/221733856620


In [33]:
# 블로그 본분, 댓글 합쳐 하나의 document로 만들기 

data['doc'] = data['review'] + ' ' + data['comment']
data.head()

,Unnamed: 0,time,review,comment,like,url,doc
0,0,2019. 1. 13. 21:35,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ...",[],8,https://blog.naver.com/14thmay/221441149660,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ..."
1,1,2019. 1. 5. 13:31,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...,[],15,https://blog.naver.com/nodaybuttoday/221434831902,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...
2,2,2019. 1. 27. 22:20,"술집 창원 애월애, 인스타 감성 술집 텔스 ...",[],4,https://blog.naver.com/seyyyoung/221452442833,"술집 창원 애월애, 인스타 감성 술집 텔스 ..."
3,5,2019. 1. 21. 17:00,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 ...,[],23,https://blog.naver.com/hongki1990/221447452932,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 ...
4,7,2019. 1. 5. 2:57,식당 여수 학동 술집 추천 / 인스타 핫플 파흐베륵 ...,[],7,https://blog.naver.com/soso__log/221434626247,식당 여수 학동 술집 추천 / 인스타 핫플 파흐베륵 ...


In [34]:
data.doc = data.doc.str.replace("[^가-힣 ]"," ")   # 한글만 남기고 특수문자, 숫자, 영어
data.doc = data.doc.str.replace("\s+", " ")         # white space삭제

data.head()

C:\Users\user\AppData\Local\Temp\ipykernel_11420\3031334446.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ")   # 한글만 남기고 특수문자, 숫자, 영어
C:\Users\user\AppData\Local\Temp\ipykernel_11420\3031334446.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ")         # white space삭제


,Unnamed: 0,time,review,comment,like,url,doc
0,0,2019. 1. 13. 21:35,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ...",[],8,https://blog.naver.com/14thmay/221441149660,합정 술집 인스타 대장 엉클조빠 합정 칵테일바 몽글 복사 이웃추가 본문 기타 기능...
1,1,2019. 1. 5. 13:31,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...,[],15,https://blog.naver.com/nodaybuttoday/221434831902,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 진작 복사 이웃추가 본문 기타 ...
2,2,2019. 1. 27. 22:20,"술집 창원 애월애, 인스타 감성 술집 텔스 ...",[],4,https://blog.naver.com/seyyyoung/221452442833,술집 창원 애월애 인스타 감성 술집 텔스 복사 이웃추가 본문 기타 기능 지도로 보...
3,5,2019. 1. 21. 17:00,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 ...,[],23,https://blog.naver.com/hongki1990/221447452932,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 승블로리 복사 이웃추가 본문...
4,7,2019. 1. 5. 2:57,식당 여수 학동 술집 추천 / 인스타 핫플 파흐베륵 ...,[],7,https://blog.naver.com/soso__log/221434626247,식당 여수 학동 술집 추천 인스타 핫플 파흐베륵 소엥이 복사 이웃추가 본문 기타 ...


In [35]:
data.doc

0         합정 술집 인스타 대장 엉클조빠 합정 칵테일바 몽글 복사 이웃추가 본문 기타 기능...
1         맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 진작 복사 이웃추가 본문 기타 ...
2         술집 창원 애월애 인스타 감성 술집 텔스 복사 이웃추가 본문 기타 기능 지도로 보...
3         냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 승블로리 복사 이웃추가 본문...
4         식당 여수 학동 술집 추천 인스타 핫플 파흐베륵 소엥이 복사 이웃추가 본문 기타 ...
                               ...                        
13392     냐미 동래 분위기 좋은 술집 주화담 강추 리밍 복사 이웃추가 본문 기타 기능 지도...
13393     먹기위해 산다 합정동 학쌀롱 분위기 좋은 합정역 술집 김도이 복사 이웃추가 본문 ...
13394     서울 성수 술집 맛도 분위기도 좋은 이자카야 용싸롱 파추만추 복사 이웃추가 본문 ...
13395     먹어 본 그 맛 이태원 라운지바 데이앤나잇 분위기 좋은 술집 사소 복사 이웃추가 ...
13396     맛집 대구 술집 대구펍 분위기 좋은 삼덕동술집 퍼스 치오 복사 이웃추가 본문 기타...
Name: doc, Length: 13397, dtype: object

In [36]:
data.doc[2]

' 술집 창원 애월애 인스타 감성 술집 텔스 복사 이웃추가 본문 기타 기능 지도로 보기 전체지도지도닫기 공유하기 신고하기 위치애월애 창원 인스타 감성 술집 애월애 요새 페북이랑 인스타에서 핫한 창원 술집애월애 다녀왔어요 위치는 창원 상남동 맥도날드 라인에 있어요 더보기 지도 데이터 지도 컨트롤러 범례부동산거리읍 면 동시 군 구시 도국가 애월애 경상남도 창원시 성산구 상남동 주소 남겨두겠습니다 메뉴판 사진 입니당 저희는 가장 인기있다는 딱새우 연어사시미 원과맥주 한병 시켰어요 기본안주는 완두콩과 단무지 내부 사진은 깜박하고 못 찍어왔어요 찍은 거라곤 이것 뿐 다들 애월애가면 심술이나 한라산시켜서 인증샷 찍어가시더라구요 대화하고 셀카도 몇 장 찍다보니저희가 주문한 딱새우랑 연어사시미가 짠 비쥬얼은 정말 인정 딱새우는 이렇게 손질되서 나오구 연어는 두툼하게 다시봐도 비쥬얼은 누구나 혹할만함 연어는 확실히 담백하고 맛있었고두툼해서 좋았어요 그리고 다양한 부위들이나와서 골라먹는 재미도 있었구요 하지만 둘이서 먹기에는 물리는 감이 셋 이상 가실 경우에 추천드려요 새우는 저한텐 비렸어요 하나 먹자마자 비려서 안 먹을까하다가혹시나하는 마음에 하나를 더 먹었는데역시나 비려서 그 이후론 입도 안댐 그래도 남자친구는 잘 먹더라구요 돌아오는 길에 둘다 속이 니글거려서집 앞 편의점에서 틈새라면 먹었어욬 '

In [37]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

In [38]:
!pip install konlpy
!pip install nltk

In [39]:
from konlpy.tag import Okt 
okt = Okt()

In [40]:
# 불용어 
stopwords = pd.read_csv('ko-stopwords.csv')   #한국어불용어사전 불러오기
stopwords = list(stopwords['stopwords'])
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",'에서','고',
                  '이다','쓰리다 도어즈','일요일','휴무','수표','층','틀다','차차','드므','맛집','술집','길','층','개다 성','사진',
                  '파다','동영상','아스 론','자다','는','희다 스토리','한','씨', "것","거","게","데","이다","건","고","되다","되어다",
                  "걸","기", "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다", 
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다", "먹다", "보다",
                  "좋다", "너무","을지로","신당", '꿀니', '복사', '이웃추가', '본문', '기타', '기능', '지도로', '보기', '더', '곳','전체',
                  '지도', '지도닫기', '공유하기', '신고하기', '위치', '사진으로', '마무리', '안녕하세요', '안녕', '술집', '맛집','끝',
                  '더보기', '지도', '데이터', '지도', '컨트롤러', '범례', '부동산', '거리', '읍', '면', '동시', '군', '구시', '도국가',
                  '랭맥','공유하기 신고하기 더보기 지도 데이터 지도 컨트롤러 범례부동산거리읍 면 동시 군 구시 도국가', '스토리', '삼발이'
                 ]) #불용어 추가

stopwords = set(stopwords)   # 중복제거

#형태소 분석 함수 만들기
def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True)   # 형태소 분석, 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag
            in ['Noun','Adjective','Verb','Adverd'] if word not in stopwords]
    return words


In [49]:
# 형태소 분석 적용
from tqdm import tqdm
tokenizing_doc = []
for i in tqdm(data['doc']):
    tokenizing_doc.append(okt_pos_tagging(i))
data['token'] = tokenizing_doc # 새로운 tokenizing이라는 컬럼에 키워드 형태로 만든 원본     

100%|████████████████████████████████████████████████████████████████████████████| 13397/13397 [30:32<00:00,  7.31it/s]


In [50]:
data.token

0        [합정, 인스타, 대장, 엉클다, 빠, 합정, 칵테일, 바, 이웃, 추가, 도지, ...
1        [서면, 추천, 개다, 성, 인스타, 감성, 분위기, 진작, 이웃, 추가, 도지, ...
2        [창원, 애월애, 인스타, 감성, 텔스, 이웃, 추가, 도지, 닫다, 공유, 신고,...
3        [이태원, 분위기, 볼레로, 인스타, 감성, 승블, 로리, 이웃, 추가, 도지, 닫...
4        [식당, 여수, 학동, 추천, 인스타, 핫, 파흐베륵, 소엥, 이웃, 추가, 도지,...
                               ...                        
13392    [냐미, 동래, 분위기, 화담, 강추, 리밍, 이웃, 추가, 도지, 닫다, 공유, ...
13393    [먹기, 위해, 산다, 합정동, 학쌀롱, 분위기, 합정역, 김도이, 이웃, 추가, ...
13394    [서울, 성수, 맛, 분위기, 이자카야, 용, 싸다, 롱, 파추, 만추, 이웃, 추...
13395    [맛, 이태원, 라운지, 바, 데이, 앤, 나잇, 분위기, 사소, 이웃, 추가, 도...
13396    [대구, 대구, 펍, 분위기, 삼덕동, 퍼스, 치다, 이웃, 추가, 도지, 닫다, ...
Name: token, Length: 13397, dtype: object

In [51]:
# 명사 추출
text_noun = [okt.nouns(doc) for doc in data.doc]
data['text_noun']  = text_noun
data['text_noun']  = data.text_noun.apply(lambda x:
                                         [a for a in x if a not in stopwords])

In [52]:
data

,Unnamed: 0,time,review,comment,like,url,doc,token,text_noun
0,0,2019. 1. 13. 21:35,"✿✿✿ 합정 술집 : 인스타 대장 ""엉클조빠"" 합정 칵테일바 ...",[],8,https://blog.naver.com/14thmay/221441149660,합정 술집 인스타 대장 엉클조빠 합정 칵테일바 몽글 복사 이웃추가 본문 기타 기능...,"[합정, 인스타, 대장, 엉클다, 빠, 합정, 칵테일, 바, 이웃, 추가, 도지, ...","[합정, 인스타, 대장, 빠, 합정, 칵테일, 바, 이웃, 추가, 도지, 공유, 신..."
1,1,2019. 1. 5. 13:31,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 ...,[],15,https://blog.naver.com/nodaybuttoday/221434831902,맛집 서면 술집 추천 갬성 인스타감성 분위기 술집 진작 복사 이웃추가 본문 기타 ...,"[서면, 추천, 개다, 성, 인스타, 감성, 분위기, 진작, 이웃, 추가, 도지, ...","[서면, 추천, 성, 인스타, 감성, 분위기, 진작, 이웃, 추가, 도지, 공유, ..."
2,2,2019. 1. 27. 22:20,"술집 창원 애월애, 인스타 감성 술집 텔스 ...",[],4,https://blog.naver.com/seyyyoung/221452442833,술집 창원 애월애 인스타 감성 술집 텔스 복사 이웃추가 본문 기타 기능 지도로 보...,"[창원, 애월애, 인스타, 감성, 텔스, 이웃, 추가, 도지, 닫다, 공유, 신고,...","[창원, 애월애, 인스타, 감성, 텔스, 이웃, 추가, 도지, 공유, 신고, 애월애..."
3,5,2019. 1. 21. 17:00,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 ...,[],23,https://blog.naver.com/hongki1990/221447452932,냠냠 이태원 분위기 좋은 술집 볼레로 인스타감성 술집 승블로리 복사 이웃추가 본문...,"[이태원, 분위기, 볼레로, 인스타, 감성, 승블, 로리, 이웃, 추가, 도지, 닫...","[이태원, 분위기, 볼레로, 인스타, 감성, 승블, 로리, 이웃, 추가, 도지, 공..."
4,7,2019. 1. 5. 2:57,식당 여수 학동 술집 추천 / 인스타 핫플 파흐베륵 ...,[],7,https://blog.naver.com/soso__log/221434626247,식당 여수 학동 술집 추천 인스타 핫플 파흐베륵 소엥이 복사 이웃추가 본문 기타 ...,"[식당, 여수, 학동, 추천, 인스타, 핫, 파흐베륵, 소엥, 이웃, 추가, 도지,...","[식당, 여수, 학동, 추천, 인스타, 핫, 파흐베륵, 소엥, 이웃, 추가, 도지,..."
...,...,...,...,...,...,...,...,...,...
13392,21230,2019. 12. 30. 23:58,냐미 동래 분위기 좋은 술집 : 주화담 강추 ! 리...,[],10,https://blog.naver.com/ksrsol/221754718804,냐미 동래 분위기 좋은 술집 주화담 강추 리밍 복사 이웃추가 본문 기타 기능 지도...,"[냐미, 동래, 분위기, 화담, 강추, 리밍, 이웃, 추가, 도지, 닫다, 공유, ...","[냐미, 동래, 분위기, 화담, 강추, 리밍, 이웃, 추가, 도지, 공유, 신고, ..."
13393,21231,2019. 12. 29. 12:37,먹기위해 산다 합정동 학쌀롱 - 분위기 좋은 합정역 술집 !!...,[],11,https://blog.naver.com/kyungbling/221753017814,먹기위해 산다 합정동 학쌀롱 분위기 좋은 합정역 술집 김도이 복사 이웃추가 본문 ...,"[먹기, 위해, 산다, 합정동, 학쌀롱, 분위기, 합정역, 김도이, 이웃, 추가, ...","[먹기, 위해, 산다, 합정동, 학쌀롱, 분위기, 합정역, 김도이, 이웃, 추가, ..."
13394,21232,2019. 12. 29. 10:46,in 서울 [성수 술집] 맛도 분위기도 좋은 이자카야 용싸롱 ...,[],11,https://blog.naver.com/soar2high/221752944616,서울 성수 술집 맛도 분위기도 좋은 이자카야 용싸롱 파추만추 복사 이웃추가 본문 ...,"[서울, 성수, 맛, 분위기, 이자카야, 용, 싸다, 롱, 파추, 만추, 이웃, 추...","[서울, 성수, 맛, 분위기, 이자카야, 용, 롱, 파추, 만추, 이웃, 추가, 도..."
13395,21237,2019. 12. 16. 18:30,먹어 본 . 그 맛 이태원 라운지바 데이앤나잇 분위기 좋은 술...,[],5,https://blog.naver.com/jjangmi95/221733856620,먹어 본 그 맛 이태원 라운지바 데이앤나잇 분위기 좋은 술집 사소 복사 이웃추가 ...,"[맛, 이태원, 라운지, 바, 데이, 앤, 나잇, 분위기, 사소, 이웃, 추가, 도...","[맛, 이태원, 라운지, 바, 데이, 앤, 나잇, 분위기, 사소, 이웃, 추가, 도..."


### clustering (tf-idf)

In [45]:
# 리스트 차원 낮춰주는 함수 생성

def flatten(l):
    flatlist = []
    for elem in l:
        if type(elem)  == list:
            for e in elem:
                flatlist.append(e)
        else:
            flatlist.append(elem)
        return flatlist

In [46]:
data1 = data.token.tolist()   # series -> list
data2 = flatten(data1)
data2[:20]

['합정',
 '인스타',
 '대장',
 '엉클다',
 '빠',
 '합정',
 '칵테일',
 '바',
 '이웃',
 '추가',
 '도지',
 '닫다',
 '공유',
 '신고',
 '엉클다',
 '빠',
 '홍대',
 '칵테일',
 '바',
 '구래']

In [47]:
text = nltk.Text(data2)
text

<Text: 합정 인스타 대장 엉클다 빠 합정 칵테일 바...>

In [48]:
data3 = []


for i in data1:
    string = i[0]
    for w in i[1:]:   # 첫번재 이후 단어부터 토큰으로 이어 붙여줌
        string += " " # 단어 사이는 띄어쓰기로 연결
        string += w   # (띄어쓰기 후 다음 단어) 써주기를 반복
    data3.append(string)
    
data3   # 쉼표를 기준으로 document가 구분됨(,)

IndexError: list index out of range

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(data3)

In [ ]:
from collections import defaultdict

word2id =  defaultdict(lambda:0)   # value가 없는 key인 경우 error 대신 0 을 출력

In [ ]:
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

In [ ]:
tfidf = []
for i,sent in enumerate(data3):
    
    tfidf.append([(token, sp_matrix[i, word2id[token]]) for token in sent.split()])

In [ ]:
tfidf[4]

## 토픽개수 정하기

In [ ]:
!pip install gensim

In [ ]:
from gensim import corpora, models

dictionary = corpora.Dictionary(data1)
dictionary

In [ ]:
# tuple로 mapping해주는 함수, doc2bow
corpus = [dictionary.doc2bow(text) for text in data1]

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
corpus[:1]

In [ ]:
values = []
for i in range(2, 10):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                              num_topics=i,
                                              id2word=dictionary)
    
    values.append(ldamodel.log_perplexity(corpus))

In [ ]:
x = range(2,10)
plt.plot(x, values)
plt.xlabel('Number of Topics')
plt.ylabel('Score')

In [ ]:
# coherence 확인 (군집 평가)
from gensim.models import CoherenceModel

coherence_values = []
for i in range(2, 10):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                              num_topics=i,
                                              id2word=dictionary)
    
    coherence_model_lda = CoherenceModel(model=ldamodel,
                                        texts = data1,
                                        dictionary=dictionary,
                                        topn=10)
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

In [ ]:
x= range(2,10)
plt.plot(x,coherence_values)
plt.xlabel('Number of Topics')
plt.ylabel('Score')

## ward clustering

In [ ]:
# 라이브러리 import

from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc

In [ ]:
sp_matrix1 = vectorizer.fit_transform(data3)

In [ ]:
df1 = pd.DataFrame(sp_matrix1.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
cluster = AgglomerativeClustering(n_clusters=6,
                                  linkage='ward')

In [ ]:
result = cluster.fit_predict(df1)

In [ ]:
df_ward = data.copy()

In [ ]:
df_ward['클러스터'] = list(result)

In [ ]:
df_ward

In [ ]:
plt.figure(figsize=(10,7))
dend = shc.dendrogram(shc.linkage(df1, method='ward'))

In [60]:
# csv로 저장
data.to_csv("2019와인(전처리).csv", encoding = "utf-8-sig")